<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/28_voila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

## Deploy Earth Engine Apps using Voila and ngrok

**Steps to deploy an Earth Engine App:**
1. Install ngrok by following the [instruction](https://ngrok.com/download)
2. Install voila by following the [instruction](https://voila.readthedocs.io/en/stable/install.html)
3. Download the notebook [28_voila.ipynb](https://github.com/giswqs/geemap/blob/master/examples/notebooks/28_voila.ipynb) 
4. Run this from the command line: `voila --no-browser 28_voila.ipynb`
5. Run this from the command line: `ngrok http 8866`
6. Copy the link from the ngrok terminal window. The links looks like the following: https://randomstring.ngrok.io
7. Share the link with anyone. 

**Optional steps:**
* To show code cells from you app, run this from the command line: `voila --no-browser --strip_sources=False 28_voila.ipynb`
* To protect your app with a password, run this: `ngrok http -auth="username:password" 8866`
* To run python simple http server in the directory, run this:`sudo python -m http.server 80` 

In [ ]:
import os
import ee
import geemap
import ipywidgets as widgets

In [ ]:
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map

In [ ]:
style = {'description_width': 'initial'}
title = widgets.Text(
    description='Title:',
    value='Landsat Timelapse',
    width=200,
    style=style
)

bands = widgets.Dropdown(
    description='Select RGB Combo:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='NIR/Red/Green',
    style=style
)

hbox1 = widgets.HBox([title, bands])
hbox1

In [ ]:
speed = widgets.IntSlider(
    description='  Frames per second:',
    tooltip='Frames per second:',
    value=10,
    min=1, 
    max = 30,
    style=style
)

cloud = widgets.Checkbox(
    value=True,
    description='Apply fmask (remove clouds, shadows, snow)',
    style=style
)

hbox2 = widgets.HBox([speed, cloud])
hbox2

In [ ]:
start_year = widgets.IntSlider(description='Start Year:', value=1984, min=1984, max=2020, style=style)
end_year = widgets.IntSlider(description='End Year:', value=2020, min=1984, max=2020, style=style)
start_month = widgets.IntSlider(description='Start Month:', value=5, min=1, max=12, style=style)
end_month = widgets.IntSlider(description='End Month:', value=10, min=1, max=12, style=style)
hbox3 = widgets.HBox([start_year, end_year, start_month, end_month])
hbox3

In [ ]:
font_size = widgets.IntSlider(description='Font size:', value=30, min=10, max=50, style=style)

font_color = widgets.ColorPicker(
    concise=False,
    description='Font color:',
    value='white',
    style=style
)

progress_bar_color = widgets.ColorPicker(
    concise=False,
    description='Progress bar color:',
    value='blue',
    style=style
)

hbox4 = widgets.HBox([font_size, font_color, progress_bar_color])
hbox4

In [ ]:
create_gif = widgets.Button(
    description='Create timelapse',
    button_style='primary',
    tooltip='Click to create timelapse',
    style=style
)

download_gif = widgets.Button(
    description='Download GIF',
    button_style='primary',
    tooltip='Click to download timelapse',
    disabled=False,
    style=style
)

output = widgets.Output()

hbox5 = widgets.HBox([create_gif])
hbox5

In [ ]:
def submit_clicked(b):
    
    with output:
        output.clear_output()
        if start_year.value > end_year.value:
            print('The end year must be great than the start year.')
            return
        if start_month.value > end_month.value:
            print('The end month must be great than the start month.')
            return        
        if start_year.value == end_year.value:
            add_progress_bar = False
        else:
            add_progress_bar = True
            
        start_date = str(start_month.value).zfill(2) + '-01'
        end_date = str(end_month.value).zfill(2) + '-30'
        
        print('Computing...')
        
        Map.add_landsat_ts_gif(roi=Map.user_roi, label=title.value, start_year=start_year.value, 
                               end_year=end_year.value, start_date=start_date, end_date=end_date, 
                               bands=bands.value.split('/'), font_color=font_color.value, 
                               frames_per_second=speed.value, font_size=font_size.value, 
                               add_progress_bar= add_progress_bar, progress_bar_color=progress_bar_color.value, 
                               download=True, apply_fmask=cloud.value) 
            
create_gif.on_click(submit_clicked)

In [ ]:
output